In [1]:
import os
import cupy as np
from keras.preprocessing import image
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.metrics import confusion_matrix, accuracy_score

In [2]:
trainpath=[]
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/Familyguy/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/Gumball/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/Tsubasa/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/adventure_time/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/catdog/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/pokemon/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/pokemon/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/smurfs/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/southpark/')
trainpath.append('../input/cartoon-classification/cartoon_classification/TRAIN/tom_and_jerry/')

testpath=[]
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/Familyguy/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/Gumball/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/Tsubasa/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/adventure_time/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/catdog/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/pokemon/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/pokemon/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/smurfs/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/southpark/')
testpath.append('../input/cartoon-classification/cartoon_classification/TEST/tom_and_jerry/')

In [3]:
columns=['Familyguy','Gumball','Tsubasa','adventure_time','catdog','pokemon','smurfs','southpark','spongebob','tom_and_jerry']

In [4]:
data=[]
label=[]
for i in range(len(trainpath)):
    trainfiles=os.listdir(trainpath[i])
    for j in range(len(trainfiles)):
        try:
            img = image.load_img(trainpath[i]+trainfiles[j],target_size=(18,32))
            vec=np.array(img)
            data.append(vec)
            label.append(i)
        except:
            print('Error in loading image '+ trainfiles[j])
    print(i)
        
train_data=np.array(data)
train_label=np.array(label)

In [30]:
train_data.shape

In [31]:
train_label.shape

In [6]:
%%time
data=[]
label=[]
for i in range(len(testpath)):
    testfiles=os.listdir(testpath[i])
    for j in range(len(testfiles)):
        try:
            img = image.load_img(testpath[i]+testfiles[j],target_size=(18,32))
            vec=np.array(img)
            data.append(vec)
            label.append(i)
        except:
            print('Error in loading image '+ testfiles[j])
    print(i)
test_data=np.array(data)
test_label=np.array(label)

In [32]:
test_data.shape

In [33]:
test_label.shape

In [7]:
test_data=test_data/255
train_data=train_data/255

# Training of model using Random Forest classifier 

In [11]:
RFC_model = cuRFC(max_features=1.0,
                   n_bins=10,
                   n_estimators=40)
RFC_model.fit(train_data.reshape((121564,1728)).astype('float32'),train_label.astype('float32'))
RFC_predict = RFC_model.predict(test_data.reshape((17761,1728)).astype('float32'))

In [12]:
accuracy_score(test_label,RFC_predict)

# Trianing of Model using Deep Learning - Convolutional Neural Networks

In [13]:
import tensorflow as tf

In [17]:
train_label_cat = tf.keras.utils.to_categorical(train_label.get(),10)
train_label_cat.shape

In [18]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape=(18,32,3)),
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

In [19]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') > 0.999):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
    
callbacks = myCallback()

In [22]:
Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.0005, 
            beta_1=0.9, 
            beta_2=0.999, 
            epsilon=1e-07,
            name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data.get(), train_label_cat, batch_size = 50, epochs = 20, callbacks=[callbacks])

In [27]:
results = model.predict(test_data.get())

# select the index with the maximum probability
results = results.argmax(axis = 1)

#results = pd.Series(results,name="Label")

In [29]:
accuracy_score(test_label,results)